In [2]:
%pip list | grep great

great-expectations        1.1.0      
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importing ge package
import great_expectations as gx
# Other Imports used
# from audit_process.snowflake_connector import SnowflakeConnection as sfc
import pandas as pd
from great_expectations.data_context import get_context
# from great_expectations.data_context import BaseDataContext
# from great_expectations.data_context.types.base import DataContextConfig, S3StoreBackendDefaults
# from great_expectations.checkpoint import SimpleCheckpoint
# from great_expectations.core.batch import RuntimeBatchRequest
# import boto3
# import yaml
# import re
# import sys
# from great_expectations.data_context.types.resource_identifiers import ValidationResultIdentifier, \
#     ExpectationSuiteIdentifier, BatchIdentifie

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp

In [11]:
# Initial Spark Session
spark = SparkSession.builder \
    .appName("TestHuditoMinIO") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .getOrCreate()

# Show Spark Context
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc

<SparkContext master=spark://spark-master:7077 appName=TestHuditoMinIO>

In [12]:
# Show HR Data Hudi Path
hudi_table_path = "s3a://warehouse/hr_silver/silver_hris_employee"

# Read HR Data
df_spark = spark.read.format("hudi").load(hudi_table_path)
df_pandas = df_spark.toPandas()
df_pandas.head()

# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


,_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,ID,FirstName,LastName,StartDate,ExitDate,...,WorkLifeBalanceScore,TrainingDate,TrainingProgramName,TrainingType,TrainingOutcome,Location,Trainer,TrainingDurationDays,TrainingCost,CurrentTimeStamp
0,20251103165530942,20251103165530942_0_0,1625,,8ae92c09-f38f-41f9-8a10-350f87ac1e5d-0_0-67-63...,1625,Blaine,Contreras,10-Jun-20,13-Nov-22,...,4,06-Nov-22,Project Management,Internal,Completed,South William,Charles Spencer,4,507.49,2025-11-03 16:55:30.967213
1,20251103165530942,20251103165530942_0_1,2470,,8ae92c09-f38f-41f9-8a10-350f87ac1e5d-0_0-67-63...,2470,Aidyn,Cuevas,04-Mar-22,02-Jul-23,...,2,26-Nov-22,Leadership Development,Internal,Passed,Andrewston,John Gallagher,2,926.17,2025-11-03 16:55:30.967213
2,20251103165530942,20251103165530942_0_2,2251,,8ae92c09-f38f-41f9-8a10-350f87ac1e5d-0_0-67-63...,2251,Presley,Dodson,30-May-21,None,...,3,06-Apr-23,Project Management,External,Failed,Mooreburgh,Alexa Brown,3,166.28,2025-11-03 16:55:30.967213
3,20251103165530942,20251103165530942_0_3,2518,,8ae92c09-f38f-41f9-8a10-350f87ac1e5d-0_0-67-63...,2518,Kamryn,Mckinney,03-Sep-19,18-Mar-22,...,1,01-Jul-23,Communication Skills,External,Failed,New Ronaldmouth,William Atkinson,5,826.82,2025-11-03 16:55:30.967213
4,20251103165530942,20251103165530942_0_4,2517,,8ae92c09-f38f-41f9-8a10-350f87ac1e5d-0_0-67-63...,2517,Annie,Galloway,10-May-19,None,...,1,17-Feb-23,Communication Skills,Internal,Passed,Charlesshire,Richard Clark,4,502.56,2025-11-03 16:55:30.967213


In [15]:
# Init the Context
context = gx.get_context()

In [16]:
data_source = context.data_sources.add_pandas("pandas")
data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df_pandas})

In [19]:
# Test Min-Max

# 'TrainingDurationDays' Check Range
expectation = gx.expectations.ExpectColumnValuesToBeBetween(
    column="TrainingDurationDays", min_value=1, max_value=5
)

# ฺData Validation Batch Result
validation_result = batch.validate(expectation)

# Print Validatoin Result
print(validation_result)

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "type": "expect_column_values_to_be_between",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column": "TrainingDurationDays",
      "min_value": 1.0,
      "max_value": 5.0
    },
    "meta": {}
  },
  "result": {
    "element_count": 3000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_counts": [],
    "partial_unexpected_index_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [18]:
# passenger_expectation = gx.expectations.ExpectColumnMaxToBeBetween(
#     column="passenger_count",
#     min_value={"$PARAMETER": "expect_passenger_max_to_be_above"},
#     max_value={"$PARAMETER": "expect_passenger_max_to_be_below"},
# )

In [10]:
spark.stop()